In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cudf as pd1
import cupy as np1
import cuml

In [3]:
df1 = pd1.read_csv('../input/how-to-get-instagram-famous/dev.csv')

In [4]:
df1.isnull().sum()


In [5]:
df1.head()


In [6]:
df1.drop(['ID'],axis=1,inplace=True)


In [7]:
df1=pd1.get_dummies(df1)

In [8]:
df1.head()

In [9]:
df1.drop(['has_new_comments_no'],axis=1,inplace = True)

In [10]:
df1.head()

In [11]:
df1.isnull().sum()
df1.shape

In [15]:

X_input = df1.iloc[:,0:24]
Y_output = df1.iloc[:,24]


In [16]:
X_input.shape

In [17]:
Y_output


In [18]:
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
X_scaled=ss.fit_transform(X_input)

In [ ]:
X_scaled=pd1.DataFrame(X_scaled)

In [19]:
X_scaled.shape

In [20]:
#train-test_split
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y_output,test_size=0.30,random_state=42)

In [21]:
# RANDOM FOREST CLASSIFIER
from cuml import RandomForestClassifier as cuRF
n_estimators = 25
max_depth = 10

In [22]:
model = cuRF( max_depth = max_depth,
              n_estimators = n_estimators,
              random_state  = 47 )
    

In [23]:
X_test.shape

In [24]:
trained_RF = model.fit ( X_train, Y_train )

predictions = model.predict ( X_test )
X_test

In [25]:
cu_score = cuml.metrics.accuracy_score( Y_test, predictions )
# sk_score = accuracy_score( asnumpy( Y_test ), asnumpy( predictions ) )

print( " cuml accuracy: ", cu_score )


In [26]:
# RIDGE REGRESSION
lamda=0.001
A=X_train.values.T.dot(X_train.values)
I=np1.identity(A.shape[0])
A1=np1.add(A,lamda*I)
B=np1.linalg.inv(A1)
C=B.dot(X_train.values.T)
beta3=C.dot(Y_train.values)

In [27]:
Y_predict=X_test.values.dot(beta3)

cu1_score = cuml.metrics.accuracy_score( Y_test, predictions )
print(cu1_score)

In [28]:
#LOGISTIC REGRESSION
from cuml.linear_model.logistic_regression import LogisticRegression

lr = LogisticRegression()

In [29]:
lr.fit(X_train, Y_train)

In [30]:
preds = lr.predict(X_test)

In [31]:
cu2_score = cuml.metrics.accuracy_score( Y_test, preds )
print(cu2_score)

In [34]:
# K-MEANS CLUSTERING ALGO
from cuml.cluster import KMeans as KMeans

kmeans = KMeans(n_clusters=3, max_iter=300, init='scalable-k-means++')

In [35]:
pred = kmeans.fit(X_scaled)

In [36]:
from cuml.metrics import adjusted_rand_score

score = adjusted_rand_score(labels_true=Y_output, 
                            labels_pred=kmeans.labels_)
print(score)


In [ ]:
# Comparing all the accuracies
# i think the best model to use is random forest classifier

In [37]:
df2 = pd1.read_csv('../input/how-to-get-instagram-famous/comp.csv')

In [38]:
df2.isnull().sum()

In [39]:
X_new = df2.iloc[:,0:12]

In [40]:
X_new.shape

In [41]:
X_new=pd1.get_dummies(X_new)
X_new.head()

In [42]:
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
X_scalednew=ss.fit_transform(pd1.DataFrame(X_new))

In [43]:
X_scalednew=pd1.DataFrame(X_scalednew)

In [44]:
output = model.predict(X_scalednew)
print(output)

In [45]:
df2.head().to_pandas()


In [46]:
df2['has_new_comments'] = output

In [47]:
df2.head()

In [48]:
df2['has_new_comments'] = df2['has_new_comments'].map(
                   {0:'no',1:'yes'})

In [49]:
df2.head()

In [50]:
df2[['ID', 'has_new_comments']]

In [51]:

submission_csv = df2.to_csv('/kaggle/working/submission.csv',columns = ['ID', 'has_new_comments'], index = False)